# Embedded Methods

## Use of Lasso(l1) and Ridge(l2) Regression Coefficient to select Features for the Model
## Classification

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.feature_selection import SelectFromModel

In [4]:
titanic = sns.load_dataset('titanic')

In [5]:
titanic.head(6)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True


In [6]:
titanic.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [7]:
titanic.drop(labels=['age', 'deck'], axis=1, inplace=True)

In [8]:
titanic.dropna(inplace=True)

In [9]:
print(titanic.shape)

(889, 13)


In [10]:
titanic.head()

,survived,pclass,sex,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,male,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,1,female,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,1,3,female,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,1,female,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,0,3,male,0,0,8.0500,S,Third,man,True,Southampton,no,True


In [11]:
data = titanic[['pclass', 'sex', 'sibsp', 'parch', 'embarked', 'who', 'alone']]

In [12]:
data.head()

,pclass,sex,sibsp,parch,embarked,who,alone
0,3,male,1,0,S,man,False
1,1,female,1,0,C,woman,False
2,3,female,0,0,S,woman,True
3,1,female,1,0,S,woman,False
4,3,male,0,0,S,man,True


In [13]:
print(data.groupby('sex').sex.count())

sex
female    312
male      577
Name: sex, dtype: int64


In [14]:
a = {'female':0, 'male':1}

In [15]:
print(data.groupby('embarked').embarked.count())

embarked
C    168
Q     77
S    644
Name: embarked, dtype: int64


In [16]:
b = {'C':0, 'Q':1, 'S':2}

In [17]:
print(data.groupby('who').who.count())

who
child     83
man      537
woman    269
Name: who, dtype: int64


In [18]:
c = {'child':0, 'man':1, 'woman':2}

In [19]:
print(data.groupby('alone').alone.count())

alone
False    354
True     535
Name: alone, dtype: int64


In [20]:
d = {True:1, False:0}

In [21]:
data['sex'] = data['sex'].map(a)
data['embarked'] = data['embarked'].map(b)
data['who'] = data['who'].map(c)
data['alone'] = data['alone'].map(d)

C:\Users\Saurav\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Saurav\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Saurav\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

In [22]:
data.head()

,pclass,sex,sibsp,parch,embarked,who,alone
0,3,1,1,0,2,1,0
1,1,0,1,0,0,2,0
2,3,0,0,0,2,2,1
3,1,0,1,0,2,2,0
4,3,1,0,0,2,1,1


In [23]:
X = data
y = titanic['survived']

In [24]:
print(X.shape)
print(y.shape)

(889, 7)
(889,)


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size = 0.1)

In [26]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(800, 7)
(800,)
(89, 7)
(89,)


### Using l1

In [27]:
## Change the values of C to get important features

In [28]:
sel = SelectFromModel(LogisticRegression(penalty='l1', C=0.5, solver = 'liblinear'))  ### C is inverse of Regularization Parameter
sel.fit(X_train, y_train)
sel.get_support()

array([ True,  True,  True,  True,  True,  True,  True])

In [29]:
X_train_l1 = sel.transform(X_train)
X_test_l1 = sel.transform(X_test)
X_train_l1.shape

(800, 7)

In [30]:
%%time
model = RandomForestClassifier(n_estimators=100, max_depth=4)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.8314606741573034
Wall time: 151 ms


In [31]:
%%time
model = RandomForestClassifier(n_estimators=100, max_depth=4)
model.fit(X_train_l1, y_train)
y_pred = model.predict(X_test_l1)
print(accuracy_score(y_test, y_pred))

0.8314606741573034
Wall time: 104 ms


In [32]:
### Using l2

In [33]:
## Change the values of C to get important features

In [34]:
sel = SelectFromModel(LogisticRegression(penalty='l2', C=0.05, solver='liblinear'))  ### C is inverse of Regularization Parameter
sel.fit(X_train, y_train)
sel.get_support()

array([ True,  True, False, False, False,  True, False])

In [35]:
X_train_l2 = sel.transform(X_train)
X_test_l2 = sel.transform(X_test)
X_train_l2.shape

(800, 3)

In [36]:
%%time
model = RandomForestClassifier(n_estimators=100, max_depth=4)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.8202247191011236
Wall time: 160 ms


In [37]:
%%time
model = RandomForestClassifier(n_estimators=100, max_depth=4)
model.fit(X_train_l2, y_train)
y_pred = model.predict(X_test_l2)
print(accuracy_score(y_test, y_pred))

0.797752808988764
Wall time: 108 ms


# Similarly we can use Ridge and Lasso to get important features for regression